In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
import matplotlib.pyplot as plt

In [1]:
# Useful functions
def IsScalar(x):
    if type(x) in (list, np.ndarray,):
        return False
    else:
        return True

    
def Thresh(x,th):
    if IsScalar(x):
        val = 1 if x>th else 0
    else:
        val = np.zeros_like(x)
        val[x>=th] = 1
    return val




def Hamming(x, y):
    d = [] #an empty list
    for xx, yy in zip(x,y):
        dd = 0.
        for xxx,yyy in zip(xx,yy):
            if xxx==1 and yyy!=1:
                dd += 1.
            elif yyy==1 and xxx!=1:
                dd += 1.
        d.append(dd)
    return d #a list of one value


def Perturb(X, p=2):
    Y = copy.deepcopy(X)
    for yy in Y:
        perm = random.sample(range(len(yy)), p)  # Randomly select p indices to flip and put them in an array(perm)
        for v in perm:
            if yy[v] == 1:
                yy[v] *= 0;
            else:
                yy[v] += 1;
    return Y


def check_binary_array(arr):
    num_ones = np.count_nonzero(arr)
    num_zeros = np.count_nonzero(arr == 0)
    
    return num_ones == num_zeros


def generate_array(n_rows, n_cols, p=0.1):
    # Compute the number of ones in each row using a binomial distribution
    ones_per_row = np.random.binomial(n_cols, p, size=n_rows)

    # Initialize the array with zeros
    arr = np.zeros((n_rows, n_cols), dtype=int)

    # Set the ones in each row
    for i, ones in enumerate(ones_per_row):
        # Randomly select the indices of the ones in the row
        ones_indices = np.random.choice(n_cols, ones, replace=False)
        arr[i, ones_indices] = 1

    # Remove any duplicate rows
    arr = np.unique(arr, axis=0)

    # Add additional rows, if necessary
    while arr.shape[0] < n_rows:
        new_rows = np.random.binomial(n_cols, p, size=(n_rows - arr.shape[0],))
        row = np.zeros(n_cols, dtype=int)
        ones_indices = np.random.choice(n_cols, new_rows, replace=False)
        row[ones_indices] = 1
        arr = np.concatenate((arr, [row]), axis=0)

    # Return the resulting array
    return arr

In [ ]:
def attenuator(inner,n,pos,att):
    b = np.zeros(n)
    b[pos] = inner[pos]*att
    return b

In [3]:
#def update(x, X, c, th):
#    y = x.copy()  # create a copy of x to avoid modifying it directly
#    dot_product = x @ X[:, c]
#    y[0, c] = Thresh(dot_product, th)  # assign the result of Thresh(y) back to the c-th element of y
#    return y

In [ ]:
def update(x, X, c, th):
    y = x.copy()  # create a copy of x to avoid modifying it directly
    dot_product = x @ X[:, c]
    y[0, c] = Thresh(dot_product, th)  # assign the result of Thresh(y) back to the c-th element of y
    return y

In [4]:
def Energy(W, x):
    E = - 0.5 * (x @ W)@x.T
    return E

In [ ]:
def num_of_cross_talk(arr):
    
    M, N = arr.shape
    sum_vector = np.sum(arr, axis=0)
    prob_of_ones = np.divide(sum_vector, M)
    #n = np.max(sum_vector)
    #for i in range(N):
    #    if (prob_of_ones[i] >= 0.5):
    #        sum_vector[i] = sum_vector[i] - 1
    #n = np.max(sum_vector)
            
    rate_of_occurrence = np.zeros(M)
    for i in range(1, M+1):
        rate_of_occurrence[i-1] = np.sum(sum_vector == i) / N
        temp = np.where(sum_vector == i, np.multiply(sum_vector, rate_of_occurrence[i-1]), sum_vector)
        sum_vector = temp


    n = np.mean(sum_vector)
    
    return n

In [ ]:
def find_thresh(matrix, m):
    n = matrix.shape[0]
    col_means = np.zeros(n)
    col_sum = 0
    cross_sum = 0
    
    avg_inner = np.sum(np.diag(matrix))/n
    
    for i in range(n):
        col_means[i] = np.mean(np.delete(matrix[:, i], i))
        col_sum += col_means[i]
    
    for i in range(int(np.round(m))):
        cross_sum += col_sum / n
        
    return cross_sum + avg_inner

In [5]:
def stable_state(v, M):
    for i in range(M.shape[0]):
        if np.array_equal(v[0], M[i]):
            return i
    else:
        print('None')